In [7]:
import numpy as np
w,b = np.array([0,0]), 0
def d(x):
    return np.dot(x,w)+b
def sign(x):
    return 1 if x >= 0 else -1
def h(x):
    return sign(d(x))
def clf_score(X,y):
    score = 0
    for xi, yi in zip(X,y):
        score += yi*h(xi)
    return score

# data
X = np.array([[5,2], [3,2], [2,7], [1,4], [6,1], [4,5]])
y = np.array([-1, -1, 1, 1, -1, 1])

# PLA-pocket
epochs = 10
best_w, best_b = 0, 0
best_cs = clf_score(X,y)
for i in range(epochs):
    for xi, yi in zip(X,y):
        if yi*d(xi)<=0:
            w = w + yi*xi
            b = b + yi
            cs = clf_score(X,y)
            if cs > best_cs:
                best_cs = cs
                best_w = w
                best_b = b
            break
best_cs,best_w,best_b
            


(6, array([-7,  6]), -1)

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import KFold
from matplotlib.colors import ListedColormap

# 超参数
epochs = 100 # 固定的迭代次数

# 参数
w, b = np.array([0, 0]), 0 # np.array 相当于定义向量

# 定义 d(x) 函数
def d(x):
    return np.dot(w,x)+b # np.dot 是向量的点积

# 定义 sign 函数
def sign(x):
    return 1 if x >= 0 else -1

# 定义 h(x) 函数
def h(x):
    return sign(d(x))

# 计算决策边界的评分
def clf_score(X, y):
    score = 0
    for xi, yi in zip(X, y):
        score += yi*h(xi)
    return score

# 感知机的口袋算法
def PLA_pocket(X, y):
    global epochs, w, b

    w, b = np.array([0, 0]), 0 # np.array 相当于定义向量
    best_w, best_b = w, b
    best_cs = clf_score(X, y)
    for _ in range(epochs):

        # 顺序遍及数据集 X
        for xi, yi in zip(X, y):
            # 如果有分错的
            if yi*d(xi) <= 0:
                # 更新法向量 lw 和 lb
                w, b = w + yi*xi, b + yi
                # 对新得到的决策边界进行评分
                cs = clf_score(X, y)
                # 如果更好，则进行更新
                if cs > best_cs:
                    best_cs = cs
                    best_w, best_b = w, b
                break

    w, b = best_w, best_b

# 以下是训练代码
# 载入iris数据集
iris = datasets.load_iris()
# 取后面100个数据，并且只取最后两个特征，以及取出对应的类别
sampleNumber = 100
X = iris.data[50:50+sampleNumber, [2,3]]
# iris 数据集的类别是0, 1, 2，为了运用我们实现的感知机算法，这里将后两个类别改为-1, 1
y = np.where(iris.target[50:50+sampleNumber] == 1, -1, 1)

# 借助 train_test_split 进行随机分割，按照 8 : 2  的比例划分为训练验证集、测试集
rs = 42
X_tv, X_test, y_tv, y_test = train_test_split(X, y, test_size=0.2, random_state=rs)
print(r'总共有 {} 个数据，其中训练验证集中有 {} 个数据，测试集中有 {} 个数据。'.format(len(X), len(X_tv), len(X_test)))

# 在 X_tv 上进行 k 折交叉沿验证
k = 10
kf = KFold(n_splits=k, random_state=rs, shuffle=True)
val_accuracy = 0
for idx, (train, val) in zip(range(k), kf.split(X_tv)):
    X_train, y_train, X_val, y_val = X_tv[train], y_tv[train], X_tv[val], y_tv[val]
    PLA_pocket(X_train, y_train)
    split_train_accuracy = 1 - (len(X_train) - clf_score(X_train, y_train))/2/len(X_train)
    split_val_accuracy = 1 - (len(X_val) - clf_score(X_val, y_val))/2/len(X_val)
    print(r'第 {} 折，训练集准确率 {:.2%} ，验证集准确率 {:.2%}'.format(idx + 1, split_train_accuracy, split_val_accuracy))
    val_accuracy += split_val_accuracy
print(r'epochs = {}，验证集准确率的平均值为 {:.2%}。'.format(epochs, val_accuracy / k))

总共有 100 个数据，其中训练验证集中有 80 个数据，测试集中有 20 个数据。
第 1 折，训练集准确率 79.17% ，验证集准确率 87.50%
第 2 折，训练集准确率 94.44% ，验证集准确率 87.50%
第 3 折，训练集准确率 95.83% ，验证集准确率 75.00%
第 4 折，训练集准确率 94.44% ，验证集准确率 100.00%
第 5 折，训练集准确率 94.44% ，验证集准确率 87.50%
第 6 折，训练集准确率 94.44% ，验证集准确率 100.00%
第 7 折，训练集准确率 94.44% ，验证集准确率 100.00%
第 8 折，训练集准确率 95.83% ，验证集准确率 87.50%
第 9 折，训练集准确率 94.44% ，验证集准确率 100.00%
第 10 折，训练集准确率 95.83% ，验证集准确率 87.50%
epochs = 100，验证集准确率的平均值为 91.25%。


(array([-7,  6]), -1, 6)